### Improving Regression Lines

In the previous section we saw how after choosing the slope and y-intercept values of a regression line, we use the root mean squared error to distill the goodness of fit into one number.  

Now we can go beyond that to find the "best fit" regression line by doing the following:
* Adjust $b$ and $m$, as these are the only things that can vary in a single-variable regression line.
* After each adjustment calculate the average squared error 
* The regression line (that is, the values of $b$ and $m$) with our smallest average squared error is our best fit line 

Let's see this technique in action.  For this example, let's imagine that our data does not include the point when x = 0. This leaves our dataset looking like the following:

In [28]:
first_show = {'x': 100, 'y': 275}
second_show = {'x': 200, 'y': 300}
third_show = {'x': 400, 'y': 700}

updated_shows = [first_show, second_show, third_show]

We again take an initial guess at slope by drawing a line between the first and last points, giving us a slope of 1.833.  And then let's just take an initial stab at $b$ by setting $b$ = 100.

In [29]:
def slope_between_two_points(first_point, second_point):
    return (second_point['y'] - first_point['y'])/(second_point['x'] - first_point['x'])

slope_between_two_points(updated_shows[0], updated_shows[2]) # 1.833

def regression_formula(x):
    return 1.833*x + 100
    # change the number 0 to different numbers, to see what happens
 

From there, we calculate the `sum_of_squared_errors`.

In [30]:
def y(x, points):
    point_at_x = list(filter(lambda point: point['x'] == x,points))[0]
    return point_at_x['y']

# calculate the squared error at a given value of x
def squared_error(x, movies):
    return (y(x, movies) - regression_formula(x))**2

def sum_of_squared_errors(points):
    squared_errors = list(map(lambda point: squared_error(point['x'], points), points))
    return sum(squared_errors)

sum_of_squared_errors(updated_shows) # 45566.69

45566.68999999997

Ok, 45,566.  Is that a good number? Who knows. Let's get a sense of this by plugging in different numbers for *b* and seeing what happens to the sum of squared error.

| b        | residual sum of squared           | 
| ------------- |:-------------:| 
| 100      |50000| 
| 110      |55989 | 
| 90      |50749 | 
|80 | 49029
|70 | 47909
|60 | 47389
| 50 | 47469

Now notice that simply by setting different numbers as $b$, we get a smaller residual sum of squares (RSS), given our value of $m$ at 1.83.  Setting $b$ to 110 produced a higher error, than at 100, so we tried moving in the other direction.  We kept moving our $b$ value lower until we set $b$ = 50, at which point our error increased from the value at 60.  So, we know that a value of $b$ between 50 and 60 produces the smallest RSS, when we set $m$ = 1.83. 

If we plot these two numbers on a chart, we get the following:

![](./cost-curve-plot.png)

So you can see visually from this that the point when b = 60 is the lowest RSS. So we start at value 100, and we can move back and forth until we get to around 60.  The check of every ten is called our *step size*.  

This technique is a called *gradient descent*.  Gradient just means *a series of successive changes* and that's what we do.  We successively change our value of our y-intercept.  We *descend*, as we descend along a cost curve.  When the value of our RSS no longer descends as we change our variable, we stop.

So our technique from the top of this lesson holds true: 

* Adjust $b$ and $m$, as these are the only things that can vary in a single-variable regression line.
* After each adjustment calculate the average squared error 
* The regression line (that is, the values of $b$ and $m$) with our smallest average squared error is our best fit line 

## Still, things are not so simple

In the graph of the cost curve above, it seems simple that we should just go to point 60.  But remember that we need to find the line with the lowest error where we are changing both m and b.  And in the future our formulas even more variables.  For example, right now we are considering show revenue purely as a function of budget, but what happens when we say it's a function of budget and length of the movie.  Our line will become more even complicated.  

The point is that it's not so easy to calculate the minimum when our cost curves become more complicated.  The best we can do is calculate the cost for a given set of points an adjust accordingly.  Gradient descent tells us how much to change our (for now) *m* and *b* values so that we are moving in the right direction.  

So that's how we have to imagine gradient descent.  We don't know our end destination of the values that produce our best line, but we do have a tool that will steer us in the correct direction until we arrive.

![](./maps-pointer.png)

In other words, take a look at the graph below.  We start off with our regression line of y = 1.83x + 100, and calculate the error.  Our best fit line could be at any other square in the grid -- and this grid runs horizontally and vertically through infinity.  So we don't want to calculate all of the values.

Instead we just want to make sure that we are taking the correct path to our best fit line.  Just get a little bit better with each step.  So assume our best fit line is really at when m = 1.9 and b = 60.  Gradient descent tells us the direction of our next step - in this case, diagonally down and to the right.  So we no longer need to calculate every potential square in the table - we just keep walking in the right direction.  

| b        |m = 1.83           | m = 1.9           | m = 2.0           |   
| ------------- |:-------------:| :-------------:| 
| 100      |53069| ? | ?
| 110      |? |  ? | ? 
| 90      |? |  ? | ?
|80 | ?| ? | ?
|70 | ? | ? | ?
|60 | ? | :) | ?
| 50 | ? | ? | ?

In the context of our m and b values, correct direction means that we are adjusting m and b ** with the correct proportions **.  In other words, if we increase b, but do not change m, then we are moving vertically down in our graph but not horizontally in the correct direction.  And if we adjust m too much in comparison to b, then we will begin to overshoot to the right.  Gradient descent has us re-adjust our proportions of m and b with each step along the table.  It is our google maps arrow always pointing us in the correct direction. 

Ok, so how do we get something that always points us in the correct direction?  Well, we call that our gradient.  The *b gradient * tells us how much we should be pointed in the b direction, and the * m gradient * tells us how much to be pointed in the m direction.  The following code represents **one step** in the correct direction.

```python
learning_rate = .0001
n = len(updated_shows)
b_gradient = 0
m_gradient = 0 
N = float(len(updated_shows))
for i in range(len(points)):
    b_gradient += -(1/n)*(error_at_point_x)
    m_gradient += -(1/n)*(error_at_point_x*x)

new_b = b_current - (learningRate * b_gradient)
new_m = m_current - (learningRate * m_gradient)
```

So note from the code above, that our b and m gradients start at zero.  In other words, we begin by assuming that we should not change b or m at all.  Then for each point, we see the error of our current regression line.  For each point, we adjust our b gradient by the error, divided by the number of points.  For the m_gradient, we adjust our m_gradient to by the error at a given point multiplied by the x value of that point.  

Finally, we update our value of b by the amount of the gradient.  Well, the amount of the gradient times by this learning rate.  The learning rate is simply to scale down the size of our update to our values - so that we don't overshoot the mark.  Remember that as long as we move in the right direction, we will get there.

So that is our our gradient descent process.  Start with an initial regression line with values of m and b.  Then for each point, calculate how the regression line fares against the actual point (that is, find the error).  Update the gradient, so that we will be moving the value in the opposite direction of the error.  And after running through all of the points, and adjusting the gradient with every point, update the overall value of b and m by their respective gradients, scaled down by a small learning rate.

Going forward, let's see our gradient descent formulas in action.  Then, we can develop some intuition as to why they work and where they came from.

### Seeing our gradient descent formulas in action

First, let's translate our step gradient process into some real live code.

In [80]:
first_show = {'x': 30, 'y': 45}
second_show = {'x': 40, 'y': 60}
third_show = {'x': 100, 'y': 150}

updated_shows = [first_show, second_show, third_show]

def step_gradient(b_current, m_current, points):
    b_gradient = 0
    m_gradient = 0
    learning_rate = .0001
    N = float(len(points))
    for i in range(0, len(points)):
        x = points[i]['x']
        y = points[i]['y']
        b_gradient += -(1/N) * (y - ((m_current * x) + b_current))
        m_gradient += -(1/N) * x * (y - ((m_current * x) + b_current))
    new_b = b_current - (learning_rate * b_gradient)
    new_m = m_current - (learning_rate * m_gradient)
    return {'b': new_b, 'm': new_m}

In [32]:
b = 0
m = 0

step_gradient(b, m, updated_shows)

{'b': 0.0085, 'm': 0.6249999999999999}

So just looking at input and output, we begin by setting $b$ and $m$ to 0, 0.  Then from our step_gradient function, we recieve new values of b and m of .0085 and .6245.  Now what we need to do, is take another step in the correct direction by calling our step gradient function with our updated values of b and m.

In [33]:
updated_b = 0.0085
updated_m = 0.6249
step_gradient(updated_b, updated_m, updated_shows)

{'b': 0.01345805, 'm': 0.9894768333333332}

Let's do this, say, 10 times.

In [34]:
# set our initial step with m and b values, and the corresponding error.
b = 0
m = 0
iterations = []
for i in range(10):
    iteration = step_gradient(b, m, updated_shows)
    # {'b': value, 'm': value}
    b = iteration['b']
    m = iteration['m']
    # update values of b and m
    iterations.append(iteration)

In [35]:
iterations

[{'b': 0.0085, 'm': 0.6249999999999999},
 {'b': 0.013457483333333336, 'm': 0.9895351666666665},
 {'b': 0.016348771640555558, 'm': 1.20215258815},
 {'b': 0.018034938763874835, 'm': 1.3261630333815368},
 {'b': 0.01901821141416974, 'm': 1.398492904819568},
 {'b': 0.019591516465717437, 'm': 1.4406797579467343},
 {'b': 0.019925705352372706, 'm': 1.4652855068756228},
 {'b': 0.020120428242875608, 'm': 1.4796369666804499},
 {'b': 0.02023380672219544, 'm': 1.4880075481368862},
 {'b': 0.020299740568747532, 'm': 1.4928897448417577}]

As you can see, our m and b values both update with each step.  Not only that, but with each step, our m and b values are updated less and less as the lines they produce have lower errors.

We can see this visually.  We'll write a method called `to_line` that takes an iterations and changes it to produce a data structure we can use for a frame in an animation. 

In [36]:
def to_line(m, b):
    initial_x = 0
    ending_x = 100
    initial_y = m*initial_x + b
    ending_y = m*ending_x + b
    return {'data': [{'x': [initial_x, ending_x], 'y': [initial_y, ending_y]}]}

frames = list(map(lambda iteration: to_line(iteration['m'], iteration['b']),iterations))
frames[0]

{'data': [{'x': [0, 100], 'y': [0.0085, 62.508499999999984]}]}

Now we can see how our regression line changes, and approaches our data, with each iteration.

In [37]:
from plotly.offline import init_notebook_mode, iplot
from IPython.display import display, HTML

init_notebook_mode(connected=True)

x_values_of_shows = list(map(lambda show: show['x'], updated_shows))
y_values_of_shows = list(map(lambda show: show['y'], updated_shows))
figure = {'data': [{'x': [0], 'y': [0]}, {'x': x_values_of_shows, 'y': y_values_of_shows, 'mode': 'markers'}],
          'layout': {'xaxis': {'range': [0, 110], 'autorange': False},
                     'yaxis': {'range': [0,160], 'autorange': False},
                     'title': 'Gradient Descent',
                     'updatemenus': [{'type': 'buttons',
                                      'buttons': [{'label': 'Play',
                                                   'method': 'animate',
                                                   'args': [None]}]}]
                    },
          'frames': frames}
iplot(figure)

In [38]:
x_values_of_shows

[30, 40, 100]

### Why the formula works

Let's take another look at our gradient formula.  And let's focus on the lines describing how each point changes the b_gradient, and changes the m_gradient.  

> Here, we'll offer an intuitive explanation for gradient descent.  It's ok if the below is confusing.  In a later lesson, we'll explain gradient descent from a graphical and mathematical perspective, which should fill in the picture as well.

```python
learning_rate = .0001
n = len(updated_shows)
b_gradient = 0
m_gradient = 0 

N = float(len(updated_shows))
for i in range(len(points)):
    b_gradient += -(1/n)*(error_at_point_x)
    m_gradient += -(1/n)*(error_at_point_x*x)

new_b = b_current - (learningRate * b_gradient)
new_m = m_current - (learningRate * m_gradient)
```

In the b_gradient line, it is essentially saying to adjust the value of b by the average error at each point.  It's the average error, as we iterate through each point adding the error at each point, and then we divide by the number of points, n.  

This expression for the b_gradient makes sense.  Remember that this is the amount that we should adjust the value of b by.  

> Now the below code, still doesn't need to be understood.  But do change the value of b, and see how the plots below adjust.

In [76]:
import plotly
from plotly import graph_objs, tools
plotly.offline.init_notebook_mode(connected=True)

fig = tools.make_subplots(rows=1)

scatter_trace = graph_objs.Scatter(
    x=list(map(lambda show: show['x'], updated_shows)),
    y=list(map(lambda show: show['y'], updated_shows)),
    mode="markers"
)

### CHANGE THIS VALUE OF B
m = 3
b = 20
regression_line = generate_regression_line(120, m, b)
fig.append_trace(scatter_trace, 1, 1)
fig['layout'].update(shapes=[regression_line])
fig['layout']['yaxis1'].update(range=[0, 175])
plotly.offline.iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]



As you change the value of *b*, the regression line moves up and down. And as you move b up and down, the change in the error per point is proportional to that change in b.  Move the b value down by 50 and the error for the first point changes by 50, the second point changes by 50, and the third point changes by 50.  Thus it makes sense that we would adjust b by the average error.   
```python 
b_gradient += -(1/n)*(error_at_point_x)
```

Notice that the for the m value, however, as you change m the error is sensitive to points with larger x values.  For example, try the value of the line with x equal to 1 and with x equal to 3.  You can see that when x = 3, the points furthest to the right have the most dramatic change in error.  For the point with x = 100, the regression line isn't even on the chart, whereas for the points closer to the y-axis the error appears less than 100 for each point.  The main thing to notice is that our error becomes more extreme with the higher our points' x-values.  So it makes sense that when changing the value of our m, we want our formula to reflect that a change in m does not influence all points equally.  A nudge with with m effects our points with higher x values, so the more points with higher x the more bang for our buck we can get with a change in m.    

```python
m_gradient += -(1/n)* (error_at_point_x*x)
```

Take one more look at our error chart from before.

| b        |m = 1.83           | m = 1.9           | m = 2.0           |   
| ------------- |:-------------:| :-------------:| 
| 100      |53069| ? | ?
| 110      |? |  ? | ? 
| 90      |? |  ? | ?
|80 | ?| ? | ?
|70 | ? | ? | ?
|60 | ? | :) | ?
| 50 | ? | ? | ?

Effectively what we are saying is that the higher the x values in a dataset, the more reduction in error we can get by adjusting our value of m and taking a step in that direction, as opposed to adjusting our value of m, our gradient formula accounts for this.

### Summary

Now the charts above, show something interesting.  Our regression line stays the same.  And our error function changes to be linear.  That is if we increase our value of *b*, we expect the average deviation to change by a similar amount.  So now imagine we take our first guess of b, and set b equal to 100.  You can see, from the plot of the average error on the right that this gives us an average error of -43.67.  So if changing our b value a certain amount, changes our average error by the same amount, we can simply decrease our b value by 43.67.

In [ ]:
error = average_error_variable(updated_shows, 1.83, b)

b = b + error
b # 56.33333333333333

So this is pretty cool, even by guessing our b value incorrectly the first time, we can simply look at the average error and make an adjustment.

### Adjusting the slope value

Now that we have gotten a sense for how we can adjust our y-intercept value, let's see if we can take a similar with adjusting our slope.  We came up with an initial guess of our slope simply by drawing a line between two of our points, and using the slope for that line.  That gave us an slope of 1.83.  Now let's see if we can improve on that. 

Ok, so we adjusted our y-intercept value by seeing how a change in the y-intercept changed our mean error.  Let's see how a change in the slope changes our mean error.

In [ ]:
ints = list(range(0, 30, 1))
m_values = list(map(lambda x: x/10.0, ints))
m_value_errors = list(map(lambda m_value: average_error_variable(updated_shows, m_value, 56.33), m_values))
m_linear_error_chart = list(zip(m_values, m_value_errors))
m_linear_error_chart[:5]

In [ ]:
m = 3.0
cost_line = generate_cost_line(errors, m)
regression_line = generate_regression_line(400, m, 56.33)
fig = make_subplots()
add_cost_function_trace(fig, m_linear_error_chart)
add_scatter_plot(updated_shows)

fig['layout'].update(shapes=[regression_line, cost_line])
fig['layout']['yaxis1'].update(range=[0, 1500])

plotly.offline.iplot(fig)

Our cost chart on the right is a little difficult to interpret, but the main thing to realize is altering our m value no longer alters our cost by something approaching an equal amount.  Now focus on the chart to the right.  What we want to consider is how points further to the right (that is, with a larger x-coordinate) influence our error, as we change the slope of the line.  

Notice that when the slope is 2.0, the point with x-value at 100 and x-value at 400, both miss the mark by say 100 or so.  Ok, go ahead and change the slope from 2.0 to say 3.0.  The error at x=100 rises by about 50 or so, but the error at point 400 rises by what, another 300?  A lot.  The takeaway point is that the change in the error as we change the slope of the line, does not influence all of the points equally.  The higher the x-value of a point, the more sensitive the error is to a changing slope.

So when we update m, we `m = m + error * x`, as we adjust our value of m not just by the average error, but also by our points' x-coordinate, as the further the x-coordinate the larger the error for a given point.  

### Summary